In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Femellistes

## Scrapping les postes de blogs

In [18]:
# Fonction pour récupérer les informations d'une page donnée
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('div', class_='blog-basic-grid--text')
        for article in articles:
            title = article.find('h1', class_='blog-title').text.strip()
            author = article.find('span', class_='blog-author').text.strip()
            date = article.find('time', class_='blog-date').text.strip()
            categories = [cat.text.strip() for cat in article.find_all('a', class_='blog-categories')]
            excerpt = article.find('div', class_='blog-excerpt').text.strip()
            read_more_link = article.find('a', class_='blog-more-link')['href']

            article_url = f"https://www.femelliste.com{read_more_link}"
            article_response = requests.get(article_url)
            if article_response.status_code == 200:
                article_soup = BeautifulSoup(article_response.text, 'html.parser')
                main_content = article_soup.find('div', {'data-content-field': 'main-content'})
                full_text = main_content.get_text(strip=True)
            else:
                full_text = "Échec de la requête pour l'article."

            writer.writerow({
                'Titre': title,
                'Auteur': author,
                'Date': date,
                'Catégories': ', '.join(categories),
                'Extrait': excerpt,
                'Texte Complet': full_text,
                'URL Complète': article_url
            })
    else:
        print(f"Échec de la requête avec le code d'état: {response.status_code}")

# Ouvrir un fichier TSV en mode écriture avec un délimiteur tab
with open('./text/femelliste_articles.tsv', 'w', newline='', encoding='utf-8') as tsvfile:
    fieldnames = ['Titre', 'Auteur', 'Date', 'Catégories', 'Extrait', 'Texte Complet', 'URL Complète']
    writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()

    base_url = "https://www.femelliste.com/articles-femellisme-feminisme"

    while True:
        current_url = f"{base_url}"
        scrape_page(current_url)

        # Vérifier la présence du lien "Billets plus anciens"
        older_button = BeautifulSoup(requests.get(current_url).text, 'html.parser').find('div', class_='older')
        
        # Vérifier si older_button existe et si un lien 'rel=next' est présent
        next_page_link = older_button.find('a', rel='next')['href'] if older_button and older_button.find('a', rel='next') else None

        # Sortir de la boucle s'il n'y a pas de lien suivant ou si l'URL ne change pas
        if not next_page_link or next_page_link == base_url:
            break

        # Mettre à jour l'URL pour la prochaine itération
        base_url = f"https://www.femelliste.com{next_page_link}"


# TRADFEM

In [21]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Fonction pour récupérer les informations d'une page donnée
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('div', class_='entry-thumbnail')

        for article in articles:
            article_url = article.find('a')['href']
            image_url = article.find('img')['src']

            article_response = requests.get(article_url)
            if article_response.status_code == 200:
                article_soup = BeautifulSoup(article_response.text, 'html.parser')

                title_element = article_soup.find('h1', class_='entry-title')
                title = title_element.text.strip() if title_element else "Titre non disponible"

                # Ajout pour récupérer la date
                date_element = article_soup.find('a', class_='entry-date')
                date = date_element.text.strip() if date_element else "Date non disponible"

                # Ajout pour récupérer l'auteur
                author_element = article_soup.find('a', class_='author')
                author = author_element.text.strip() if author_element else "Auteur non disponible"

                main_content = article_soup.find('div', {'id': 'content', 'class': 'site-content', 'role': 'main'})
                full_text = main_content.get_text(strip=True)
            else:
                full_text = "Échec de la requête pour l'article."

            writer.writerow({
                'Titre': title,
                'Date': date,
                'Auteur': author,
                'Image URL': image_url,
                'Texte Complet': full_text,
                'URL Complète': article_url
            })

            # Incrémenter le compteur
            global blog_count
            blog_count += 1

    else:
        print(f"Échec de la requête avec le code d'état: {response.status_code}")

# Ouvrir un fichier TSV en mode écriture avec un délimiteur tab dans le répertoire "./text/"
output_file_path = './text/tradfem_articles.tsv'
with open(output_file_path, 'w', newline='', encoding='utf-8') as tsvfile:
    fieldnames = ['Titre', 'Date', 'Auteur', 'Image URL', 'Texte Complet', 'URL Complète']
    writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()

    base_url = "https://tradfem.wordpress.com/"
    blog_count = 0

    while True:
        scrape_page(base_url)

        # Vérifier la présence du lien "Billets plus anciens"
        older_button = BeautifulSoup(requests.get(base_url).text, 'html.parser').find('div', class_='nav-previous')
        next_page_link = older_button.find('a')['href'] if older_button and older_button.find('a') else None

        # Sortir de la boucle s'il n'y a pas de lien suivant
        if not next_page_link:
            break

        # Mettre à jour l'URL pour la prochaine itération
        base_url = next_page_link

# Print le nombre de blogs scannés
print(f"Nombre de blogs scannés : {blog_count}")


KeyboardInterrupt: 